# Building Custom Algorithms

In [ ]:
import sagemaker 
import boto3
from sagemaker import get_execution_role 

role = get_execution_role()

In [ ]:
from sagemaker.local import LocalSession

session = LocalSession()
session.config = {'local': {'local_code': True}}

In [ ]:
container="581320662326.dkr.ecr.us-east-1.amazonaws.com/chap02_python:1"

In [ ]:
from sagemaker.inputs import TrainingInput

train = TrainingInput(training_s3_input_location, content_type="text/csv")

In [ ]:
estimator = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    base_job_name="abcdefghij",
    instance_type='ml.m5.large',
    output_path=training_s3_output_location,
    sagemaker_session=sagemaker.Session())

In [ ]:
estimator.set_hyperparameters(a=1, b=2, c=3)

In [ ]:
estimator.fit({'train': train})

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium', 
    endpoint_name="custom-local-py-endpoint")

In [ ]:
predictor.predict("1")
predictor.delete_endpoint()

#### Define the PATHS constant and the get_path() function. The get_path() function will be handy in helping us manage the paths and locations of the primary files and directories used in the script:

In [ ]:
import numpy as np

from flask import Flask
from flask import Response
from flask import request

from joblib import dump, load
app = Flask(__name__)

In [ ]:
PATHS = {
    'hyperparameters': 'input/config/hyperparameters.json',
    'input': 'input/config/inputdataconfig.json',
    'data': 'input/data/',
    'model': 'model/'
}

def get_path(key):
    return '/opt/ml/' + PATHS[key]

In [ ]:
def load_json(target_file):
  output = None
  with open(target_file) as json_data:
    output = json.load(json_data)

  return output

def print_json(target_json):
  pprint.pprint(target_json, indent=4)

In [ ]:
@app.route("/invocations", methods=["POST"])
def predict():
    model = load_model()
    
    post_body = request.get_data().decode("utf-8")
    
    payload_value = float(post_body)
    
    X_test = np.array([payload_value]).reshape(-1, 1)
    y_test = model.predict(X_test)
    
    return Response(response=str(y_test[0]), status=200)


@app.route("/ping")
def ping():
    return Response(response="OK", status=200)


app.run(host="0.0.0.0", port=8080)